# 🏦 Financial Market Research Agent with Bing Grounding 📈

Welcome to our **Financial Market Research Agent with Bing Grounding** tutorial! In this notebook, we'll demonstrate how to:

1. **Initialize** a project using Microsoft Foundry.
2. **Create an Agent** with the **BingGroundingTool** for real-time web search.
3. **Ask real-world questions** about market trends, interest rates, and financial news.
4. **Retrieve and display** answers with real-time market data and appropriate disclaimers.

### ⚠️ Important Model Support Note ⚠️
> Bing grounding is currently only supported in certain Azure OpenAI models (e.g. `gpt-4o-0513`).
> 
> Make sure you specify a supported model and set the `"x-ms-enable-preview": "true"` header.

## Prerequisites
- Grounding with Bing connection in Microsoft Foundry, which has to be provisioned from Azure portal.
See ["Setup Bing Grounding"](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup) in the documentation for full details.

<img src="./seq-diagrams/bing-connection.png" width="75%"/>

- A `.env` file in the parent directory containing:
  ```bash
  AI_FOUNDRY_PROJECT_ENDPOINT=<your-ai-foundry-project-endpoint>
  AZURE_AI_MODEL_DEPLOYMENT_NAME=<supported-model>
  GROUNDING_WITH_BING_CONNECTION_NAME=<the-name-of-your-bing-connection>
  ```

## Let's Explore Real-Time Financial Research!
We'll integrate **Grounding with Bing** search results into our agent so it can gather current market data, interest rate news, and economic trends. This is perfect for financial advisors who need up-to-date information! 🎉

<br/>



## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initial Setup
We'll load environment variables from `.env` and initialize our **AIProjectClient** to manage agents.

In [ ]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
    BingGroundingAgentTool,
    BingGroundingSearchToolParameters,
    BingGroundingSearchConfiguration,
)

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir.parent / '.env')

# Get tenant ID and connection string
tenant_id = os.environ.get("TENANT_ID")
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize AIProjectClient with simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )
    print("✅ Successfully initialized AIProjectClient")
    
except Exception as e:
    print(f"❌ Error initializing project client: {e}")
    print("💡 Please complete the browser authentication prompt that should appear")

## 2. Create Bing-Grounded Financial Research Agent 🌐
We'll fetch our Bing connection from AI Foundry and use `BingGroundingTool` to let our agent search for real-time market news. Then we'll create an agent that can research current interest rates, market trends, and economic news.

Make sure your `AZURE_AI_MODEL_DEPLOYMENT_NAME` is set to a Bing-supported model (for example, `gpt-4o-0513`) and that you add the header `{"x-ms-enable-preview": "true"}`.

In [ ]:
def create_bing_grounded_agent():
    """Create a financial research agent that can use Bing for real-time market data."""
    try:
        # 1. Retrieve Bing connection from your AI Foundry project
        bing_conn_name = os.environ.get("GROUNDING_WITH_BING_CONNECTION_NAME")
        if not bing_conn_name:
            print("⚠️ GROUNDING_WITH_BING_CONNECTION_NAME not set in .env - proceeding without Bing grounding")
            print("💡 This agent will work but won't have real-time market data capabilities")
            # Create agent without Bing grounding using NEW API
            agent = project_client.agents.create_version(
                agent_name="financial-research-agent-no-bing",
                definition=PromptAgentDefinition(
                    model=os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),
                    instructions="""
                        You are a financial market research assistant.
                        Always:
                        1. Provide disclaimers that you are not a licensed financial advisor.
                        2. Encourage professional consultation for investment decisions.
                        3. Provide general market information based on your training data.
                        4. Note that you don't have access to real-time market data.
                    """
                )
            )
            print(f"🎉 Created basic financial research agent (no Bing) - name: {agent.name}, version: {agent.version}")
            return agent

        try:
            bing_connection = project_client.connections.get(name=bing_conn_name)
            conn_id = bing_connection.id
            print(f"🔗 Bing Connection ID: {conn_id}")
        except Exception as e:
            print(f"❌ Error getting Bing connection '{bing_conn_name}': {e}")
            print("💡 Check if the Bing connection exists in your Microsoft Foundry project")
            try:
                connections = project_client.connections.list()
                print("Available connections:")
                for conn in connections:
                    print(f"  - {conn.name} (ID: {conn.id})")
            except Exception as list_error:
                print(f"  Could not list connections: {list_error}")
            return None

        # 2. Create Bing grounding tool using proper model classes
        bing_tool = BingGroundingAgentTool(
            bing_grounding=BingGroundingSearchToolParameters(
                search_configurations=[
                    BingGroundingSearchConfiguration(
                        project_connection_id=conn_id
                    )
                ]
            )
        )

        # 3. Create financial research agent with Bing
        agent = project_client.agents.create_version(
            agent_name="financial-market-research-agent",
            definition=PromptAgentDefinition(
                model=os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),
                instructions="""
                    You are a financial market research assistant with Bing search capabilities.
                    Always:
                    1. Provide disclaimers that you are not a licensed financial advisor and this is not investment advice.
                    2. Encourage professional consultation for investment decisions.
                    3. Use Bing to find current interest rates, market news, and economic trends.
                    4. Provide brief, accurate summaries with source citations.
                    5. Include relevant sources and publication dates for market data.
                    6. Never recommend specific investments or predict market movements.
                """,
                tools=[bing_tool]
            )
        )

        print(f"🎉 Created financial research agent with Bing (name: {agent.name}, version: {agent.version})")
        return agent
        
    except Exception as e:
        print(f"❌ Error creating Bing-grounded agent: {e}")
        return None

# Create our Bing-based financial research agent
bing_agent = create_bing_grounded_agent()

## 3. Researching Financial Markets 💬
We'll create conversation threads for financial research queries, letting the agent search Bing for current market data. We'll ask about interest rates, Fed policy, and market trends.

In [ ]:
bing_responses = []

def ask_financial_research_question(agent, user_query, conversation_id=None):
    """Ask a financial research question using the new Foundry responses API."""
    try:
        # Get OpenAI client for the new API
        openai_client = project_client.get_openai_client()
        
        # Create a conversation if not provided
        if not conversation_id:
            conversation = openai_client.conversations.create()
            conversation_id = conversation.id
            print(f"📝 Created a new conversation, ID: {conversation_id}")

        print(f"📨 Researching: '{user_query}'")

        # Use the NEW responses API with agent reference
        response = openai_client.responses.create(
            conversation=conversation_id,
            extra_body={
                "agent": {
                    "type": "agent_reference",
                    "name": agent.name,
                    "version": agent.version
                }
            },
            input=user_query
        )
        
        print(f"🤖 Research completed!\n")
        if response.output_text:
            print(f"{response.output_text}\n")
            print("-" * 50 + "\n")

        return conversation_id, response
    except Exception as e:
        print(f"❌ Error with financial research: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if bing_agent:
    # Financial research questions for FSI customers
    questions = [
        "What are the current Federal Reserve interest rate decisions and their impact on mortgage rates?",
        "What are the latest trends in banking sector performance and major bank stock movements?",
        "What are financial experts saying about inflation forecasts for the upcoming year?"
    ]

    for q in questions:
        conv_id, response = ask_financial_research_question(bing_agent, q)
        if conv_id and response:
            bing_responses.append((conv_id, response))

## 4. Cleanup & Best Practices
You can optionally delete the agent once you're done. In production, you might keep it around for repeated usage.

### Best Practices
1. **Accuracy** – Bing search results may include disclaimers or partial info. Encourage verification with credible sources.
2. **Bing Query Display** – For compliance with Bing's use and display requirements, show both **website URLs** (in the agent's response) and **Bing search query URLs** (shown above). If the model includes citations, display them as well.
3. **Limits** – Keep an eye on usage, rate limits, or policy constraints for Bing.
4. **Privacy** – Filter search queries to avoid sending sensitive data.
5. **Evaluations** – Use `azure-ai-evaluation` for iterative improvement.


In [ ]:
def cleanup_bing_agent(agent):
    """Clean up agent using new Foundry API"""
    if agent:
        try:
            # Use delete_version for the new API with correct parameter name
            project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
            print(f"🗑️ Deleted Bing-grounded agent: {agent.name} (Version: {agent.version})")
        except Exception as e:
            print(f"❌ Error cleaning up agent: {e}")
    else:
        print("No agent to clean up.")

# Uncomment if you want to remove the agent now
cleanup_bing_agent(bing_agent)

# Congratulations! 🎉

You've successfully completed the **Health & Fitness Agent with Bing Grounding** tutorial! Here's what was accomplished:

## ✅ **What We Built**

### **🌐 Fully Functional Bing-Grounded Agent**
- Created a health and fitness agent with **real Bing grounding** capabilities
- Successfully connected to Microsoft Foundry Bing connection
- Agent can now search the web for **real-time health and fitness information**
- Configured health-focused instructions with appropriate medical disclaimers

### **🔧 Key Features Demonstrated**

1. **🔗 Successful Bing Connection**
   - **Connected to actual Bing service**: Retrieved connection from Microsoft Foundry
   - **Real web search capabilities**: Agent can now access current information
   - **Proper configuration**: Fixed the `search_configurations` array format
   - **Working end-to-end**: From connection retrieval to agent responses

2. **💬 Advanced Question Processing**
   - Successfully processed health and fitness queries with **real-time data**:
     - HIIT workout trends (with current information)
     - WHO sugar intake recommendations (latest guidelines)  
     - Intermittent fasting for weight management (recent research)
   - **Live web search**: Agent searches Bing for up-to-date information
   - **Source citations**: Agent provides references from web searches

3. **📋 Enhanced Response Quality**
   - Agent provides health advice enhanced with **current web information**
   - **Real-time accuracy**: Responses reflect latest research and guidelines
   - **Source transparency**: Web search results and citations included
   - **Medical disclaimers**: Appropriate health advice disclaimers maintained

4. **🧹 Complete Resource Management**
   - Properly cleaned up the Bing-grounded agent
   - Demonstrated responsible resource management for production use

## 🎯 **Key Concepts Mastered**

- **Bing Grounding**: Successfully implemented real-time web search capabilities
- **Connection Management**: Proper retrieval and use of Microsoft Foundry connections
- **Tool Configuration**: Correct array format for search configurations
- **Real-time Information**: Agent can now access current web information
- **Source Citation**: Web search results properly integrated into responses
- **Production Readiness**: Robust error handling and resource management

## 🔍 **API Methods Mastered**

- `project_client.connections.get(name=connection_name)` - Retrieving connections correctly
- `project_client.connections.list()` - Listing available connections for debugging
- **Bing tool configuration** - Proper `search_configurations` array format
- **Real-time web search** - Agent integration with live Bing search results

## 💡 **Best Practices Recap**

1. **Environment Variables** - Use the correct variable names from your `.env` file
2. **API Parameters** - Use `name=` parameter for connection retrieval
3. **Tool Configuration** - Ensure `search_configurations` is a proper array
4. **Connection Testing** - List available connections when debugging
5. **Resource Management** - Always clean up agents and resources
6. **Health Disclaimers** - Maintain medical advice disclaimers even with real-time data
7. **Source Transparency** - Leverage web search citations for credibility

## 🔧 **Troubleshooting Guide**

**If Bing grounding fails:**
1. ✅ Check `GROUNDING_WITH_BING_CONNECTION_NAME` in `.env` file
2. ✅ Use `name=` parameter in `connections.get()` call
3. ✅ Ensure `search_configurations` is an array with exactly 1 element
4. ✅ Verify the Bing connection exists in Microsoft Foundry
5. ✅ List available connections for debugging

**For connection errors:**
1. ✅ Use `project_client.connections.list()` to see available connections
2. ✅ Verify connection IDs match expected format
3. ✅ Check Microsoft Foundry portal for connection status

## 🌟 **Major Achievement**

This notebook now demonstrates **fully functional Bing grounding** with real web search capabilities! The agent can:
- ✅ **Search the web in real-time** for current health information
- ✅ **Provide up-to-date responses** based on latest research and guidelines  
- ✅ **Include source citations** from web search results
- ✅ **Maintain health disclaimers** while leveraging current information

---

*Happy (grounded) agent building!* 🌐🤖